In [3]:
import random
import yaml
from abc import ABC
print(yaml.__version__)

5.1.2


In [5]:
Levels = {'levels':[]}
_map = EasyLevel.Map()
_obj = EasyLevel.Objects()
Levels['levels'].append({'map': _map, 'obj': _obj})

_map = MediumLevel.Map()
_obj = MediumLevel.Objects()
_obj.config = {'enemy':['rat']}
Levels['levels'].append({'map': _map, 'obj': _obj})

_map = HardLevel.Map()
_obj = HardLevel.Objects()
_obj.config = {'enemy': ['rat', 'snake', 'dragon'], 'enemy_count': 10}
Levels['levels'].append({'map': _map, 'obj': _obj})


In [6]:
Levels

{'levels': [{'map': <__main__.EasyLevel.Map at 0x20ef734e9c8>,
   'obj': <__main__.EasyLevel.Objects at 0x20ef734ebc8>},
  {'map': <__main__.MediumLevel.Map at 0x20ef734e508>,
   'obj': <__main__.MediumLevel.Objects at 0x20ef734e888>},
  {'map': <__main__.HardLevel.Map at 0x20ef7379908>,
   'obj': <__main__.HardLevel.Objects at 0x20ef7379e08>}]}

In [13]:
Levels

{'levels': [{'map': <__main__.EasyLevel.Map at 0x2b6e313dd88>,
   'obj': <__main__.EasyLevel.Objects at 0x2b6e313d0c8>},
  {'map': <__main__.MediumLevel.Map at 0x2b6e313e088>,
   'obj': <__main__.MediumLevel.Objects at 0x2b6e313e4c8>},
  {'map': <__main__.HardLevel.Map at 0x2b6e313d7c8>,
   'obj': <__main__.HardLevel.Objects at 0x2b6e313d8c8>}]}

Вам необходимо отредактировать третий блок кода (добавить возможность создания объектов с помощью yaml). Во втором блоке кода описаны действия по созданию объектов, из него можно понять какие объекты должны создаваться и какой тип данных должен быть возвращен. В первом блоке кода указана строка и способ создания объектов с использованием yaml. Ваше решение должно возвращать одинаковый результат при выполнении первого и второго блока.

In [117]:
# добавить в реализацию класса нужного нам типа данных (в код нашего класса),
# атрибут класса yaml_tag и метод класса с названием from_yaml
class AbstractLevel(yaml.YAMLObject):

    @classmethod
    def get_map(cls):
        return cls.Map()

    @classmethod
    def get_objects(cls):
        return cls.Objects()
# # def get_report(loader, node):
#             data = loader.construct_mapping(node)
# #             print('data from get_report:')
# #             print(data)
#             rep = Class.make_report(data["title"])
#             rep.filename = data["filename"]
#             # на данный момент data["parts"] пуст. Он будет заполнен позже, соответствующим обработчиком,
#             # сохраняем на него ссылку, дополнив сразу частями из rep.parts
#             data["parts"].extend(rep.parts)
#             rep.parts = data["parts"]
#             return rep

    
    @classmethod
    def from_yaml(Class, loader, node):

        def create_easy_level(loader, node): 
            _map = EasyLevel.Map()
            _obj = EasyLevel.Objects()
            return {'map': _map, 'obj': _obj}

        def create_medium_level(loader, node): 
            data = loader.construct_mapping(node)
#             print(data)
#             print(data['enemy']) # здесь не напечатается, но в объекты передается
            _map = MediumLevel.Map()
            _obj = MediumLevel.Objects()
            _obj.config = data
            return {'map': _map, 'obj': _obj}

       
        def create_hard_level(loader, node):
            data = loader.construct_mapping(node)
            _map = HardLevel.Map()
            _obj = HardLevel.Objects()
            _obj.config = data
            return {'map': _map, 'obj': _obj}
        
        loader.add_constructor(u"!easy_level", create_easy_level)
        loader.add_constructor(u"!medium_level", create_medium_level)
        loader.add_constructor(u"!hard_level", create_hard_level)
        
#         print('loader.construct_mapping(node): ', loader.construct_mapping(node))
        return loader.construct_mapping(node)
#         return Levels #str, repr?
        

    class Map(ABC):# а зачем мы наследуем здесь от abc? возможно, не надо
        pass

    class Objects(ABC):
        pass


class EasyLevel(AbstractLevel):
    
    yaml_tag = u'!easy_level' 
    
    class Map:
        def __init__(self):
            self.Map = [[0 for _ in range(5)] for _ in range(5)]
            for i in range(5):
                for j in range(5):
                    if i == 0 or j == 0 or i == 4 or j == 4:
                        self.Map[j][i] = -1  # граница карты
                    else:
                        self.Map[j][i] = random.randint(0, 2)  # случайная характеристика области

        def get_map(self):
            return self.Map

    class Objects:
        def __init__(self):
            self.objects = [('next_lvl', (2, 2))]
            self.config = {}

        def get_objects(self, _map):
            for obj_name in ['rat']:
                coord = (random.randint(1, 3), random.randint(1, 3))
                intersect = True
                while intersect:
                    intersect = False
                    for obj in self.objects:
                        if coord == obj[1]:
                            intersect = True
                            coord = (random.randint(1, 3), random.randint(1, 3))

                self.objects.append((obj_name, coord))

            return self.objects


class MediumLevel(AbstractLevel):
    yaml_tag = '!medium_level'
  
    class Map:
        def __init__(self):
            self.Map = [[0 for _ in range(8)] for _ in range(8)]
            for i in range(8):
                for j in range(8):
                    if i == 0 or j == 0 or i == 7 or j == 7:
                        self.Map[j][i] = -1  # граница карты
                    else:
                        self.Map[j][i] = random.randint(0, 2)  # случайная характеристика области

        def get_map(self):
            return self.Map

    class Objects:
        def __init__(self):
            self.objects = [('next_lvl', (4, 4))]
            self.config = {'enemy': []}

        def get_objects(self, _map):
            print(f'medium, object: {self.config}')
            for obj_name in self.config['enemy']:
                coord = (random.randint(1, 6), random.randint(1, 6))
                intersect = True
                while intersect:
                    intersect = False
                    for obj in self.objects:
                        if coord == obj[1]:
                            intersect = True
                            coord = (random.randint(1, 6), random.randint(1, 6))

                self.objects.append((obj_name, coord))

            return self.objects


class HardLevel(AbstractLevel):
    yaml_tag = '!hard_level'
    
    class Map:
        def __init__(self):
            self.Map = [[0 for _ in range(10)] for _ in range(10)]
            for i in range(10):
                for j in range(10):
                    if i == 0 or j == 0 or i == 9 or j == 9:
                        self.Map[j][i] = -1  # граница карты :: непроходимый участок карты
                    else:
                        self.Map[j][i] = random.randint(-1, 8)  # случайная характеристика области

        def get_map(self):
            return self.Map

    class Objects:
        def __init__(self):
            self.objects = [('next_lvl', (5, 5))]
            self.config = {'enemy_count': 5, 'enemy': []}

        def get_objects(self, _map):
            print(f"self.config['enemy']: {self.config['enemy']}, self.config['enemy_count']: {self.config['enemy_count']}")
            for obj_name in self.config['enemy']:
                for tmp_int in range(self.config['enemy_count']):
                    coord = (random.randint(1, 8), random.randint(1, 8))
                    intersect = True
                    while intersect:
                        intersect = False
                        if _map[coord[0]][coord[1]] == -1:
                            intersect = True
                            coord = (random.randint(1, 8), random.randint(1, 8))
                            continue
                        for obj in self.objects:
                            if coord == obj[1]:
                                intersect = True
                                coord = (random.randint(1, 8), random.randint(1, 8))

                    self.objects.append((obj_name, coord))

            return self.objects


In [120]:
Levels = yaml.load('''

levels:
- !easy_level {}
- !medium_level
    enemy: ['rat']
- !hard_level
    enemy:
        - rat
        - snake
        - dragon
    enemy_count: 10
''', Loader = yaml.Loader)

In [121]:
Levels

{'levels': [{'map': <__main__.EasyLevel.Map at 0x20ef739fd48>,
   'obj': <__main__.EasyLevel.Objects at 0x20ef750de88>},
  {'map': <__main__.MediumLevel.Map at 0x20ef750dec8>,
   'obj': <__main__.MediumLevel.Objects at 0x20ef750da88>},
  {'map': <__main__.HardLevel.Map at 0x20ef750d488>,
   'obj': <__main__.HardLevel.Objects at 0x20ef750db08>}]}

In [116]:
Levels['levels'][1]['obj'].get_objects(Levels['levels'][1]['map'])

medium, object: {'enemy': ['сat']}


[('next_lvl', (4, 4)), ('сat', (4, 3))]

In [124]:
Levels['levels'][2]['obj'].get_objects(Levels['levels'][2]['map'].get_map())

self.config['enemy']: ['rat', 'snake', 'dragon'], self.config['enemy_count']: 10


[('next_lvl', (5, 5)),
 ('rat', (4, 6)),
 ('rat', (4, 1)),
 ('rat', (2, 6)),
 ('rat', (1, 5)),
 ('rat', (5, 8)),
 ('rat', (4, 7)),
 ('rat', (5, 3)),
 ('rat', (1, 4)),
 ('rat', (8, 2)),
 ('rat', (5, 2)),
 ('rat', (4, 8)),
 ('rat', (3, 4)),
 ('rat', (6, 2)),
 ('rat', (6, 7)),
 ('snake', (2, 3)),
 ('snake', (6, 3)),
 ('snake', (3, 5)),
 ('snake', (3, 1)),
 ('snake', (1, 1)),
 ('snake', (7, 6)),
 ('snake', (7, 7)),
 ('snake', (5, 6)),
 ('snake', (8, 4)),
 ('snake', (8, 1)),
 ('dragon', (8, 8)),
 ('dragon', (8, 6)),
 ('dragon', (8, 3)),
 ('dragon', (4, 4)),
 ('dragon', (7, 1)),
 ('dragon', (4, 3)),
 ('dragon', (6, 4)),
 ('dragon', (3, 2)),
 ('dragon', (1, 6)),
 ('dragon', (4, 2))]